# ನಾಮಿತ ಘಟಕ ಗುರುತಿಸುವಿಕೆ (NER)

ಈ ನೋಟ್ಬುಕ್ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ನಿಂದ ಬಂದಿದೆ.

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) ಡೇಟಾಸೆಟ್‌ನ ಮೇಲೆ NER ಮಾದರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುವುದನ್ನು ಕಲಿಯುತ್ತೇವೆ. ಮುಂದುವರಿಯುವ ಮೊದಲು, ದಯವಿಟ್ಟು [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ಫೈಲ್ ಅನ್ನು ಪ್ರಸ್ತುತ ಡೈರೆಕ್ಟರಿಯಲ್ಲಿ ಡೌನ್‌ಲೋಡ್ ಮಾಡಿ.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## ಡೇಟಾಸೆಟ್ ಸಿದ್ಧಪಡಿಸುವುದು

ನಾವು ಡೇಟಾಸೆಟ್ ಅನ್ನು ಡೇಟಾಫ್ರೇಮ್‌ಗೆ ಓದುವುದರಿಂದ ಪ್ರಾರಂಭಿಸೋಣ. ಪ್ಯಾಂಡಾಸ್ ಬಳಕೆ ಬಗ್ಗೆ ಇನ್ನಷ್ಟು ತಿಳಿದುಕೊಳ್ಳಲು, ನಮ್ಮ [ಡೇಟಾ ಸೈನ್ಸ್ ಫಾರ್ ಬೆಗಿನರ್ಸ್](http://aka.ms/datascience-beginners) ನಲ್ಲಿ ಇರುವ [ಡೇಟಾ ಪ್ರಕ್ರಿಯೆ ಪಾಠ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) ಅನ್ನು ಭೇಟಿ ಮಾಡಿ.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


ನಾವು ವಿಶಿಷ್ಟ ಟ್ಯಾಗ್‌ಗಳನ್ನು ಪಡೆಯೋಣ ಮತ್ತು ಟ್ಯಾಗ್‌ಗಳನ್ನು ವರ್ಗ ಸಂಖ್ಯೆಗಳಾಗಿ ಪರಿವರ್ತಿಸಲು ಬಳಸಬಹುದಾದ ಲುಕ್‌ಅಪ್ ಡಿಕ್ಷನರಿಗಳನ್ನು ರಚಿಸೋಣ:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

ಈಗ ನಾವು ಅದೇ ಕೆಲಸವನ್ನು ಶಬ್ದಸಂಪದೆಯೊಂದಿಗೆ ಮಾಡಬೇಕಾಗಿದೆ. ಸರಳತೆಗೆ, ನಾವು ಶಬ್ದಗಳ ಆವರ್ತನೆಯನ್ನು ಪರಿಗಣಿಸದೆ ಶಬ್ದಸಂಪದೆಯನ್ನು ರಚಿಸುವೆವು; ನಿಜ ಜೀವನದಲ್ಲಿ ನೀವು Keras ವೆಕ್ಟರೈಜರ್ ಅನ್ನು ಬಳಸಬಹುದು ಮತ್ತು ಶಬ್ದಗಳ ಸಂಖ್ಯೆಯನ್ನು ಮಿತಿಗೊಳಿಸಬಹುದು.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

ನಾವು ತರಬೇತಿಗಾಗಿ ವಾಕ್ಯಗಳ ಡೇಟಾಸೆಟ್ ರಚಿಸಬೇಕಾಗಿದೆ. ಮೂಲ ಡೇಟಾಸೆಟ್ ಮೂಲಕ ಲೂಪ್ ಮಾಡಿ ಎಲ್ಲಾ ವೈಯಕ್ತಿಕ ವಾಕ್ಯಗಳನ್ನು `X` (ಪದಗಳ ಪಟ್ಟಿ) ಮತ್ತು `Y` (ಟೋಕನ್ ಪಟ್ಟಿ) ಎಂದು ವಿಭಜಿಸೋಣ:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


ನಾವು ಈಗ ಎಲ್ಲಾ ಪದಗಳು ಮತ್ತು ಟೋಕನ್‌ಗಳನ್ನು ವೆಕ್ಟರೈಸ್ ಮಾಡುತ್ತೇವೆ:


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

ಸರಳತೆಗೆ, ನಾವು ಎಲ್ಲಾ ವಾಕ್ಯಗಳನ್ನು ಗರಿಷ್ಠ ಉದ್ದಕ್ಕೆ 0 ಟೋಕನ್‌ಗಳೊಂದಿಗೆ ಪ್ಯಾಡ್ ಮಾಡುತ್ತೇವೆ. ನಿಜ ಜೀವನದಲ್ಲಿ, ನಾವು ಹೆಚ್ಚು ಚತುರವಾದ ತಂತ್ರವನ್ನು ಬಳಸಬಹುದು ಮತ್ತು ಪ್ಯಾಡ್ ಸರಣಿಗಳನ್ನು ಒಂದೇ ಮಿನಿಬ್ಯಾಚ್ ಒಳಗೆ ಮಾತ್ರ ಮಾಡಬಹುದು.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## ಟೋಕನ್ ವರ್ಗೀಕರಣ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವುದು

ಟೋಕನ್ ವರ್ಗೀಕರಣಕ್ಕಾಗಿ ನಾವು ಎರಡು-ಮಟ್ಟದ ದ್ವಿಮುಖ LSTM ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ಬಳಸಲಿದ್ದೇವೆ. ಕೊನೆಯ LSTM ಲೇಯರ್‌ನ ಪ್ರತಿಯೊಂದು ಔಟ್‌ಪುಟ್‌ಗೆ ಡೆನ್ಸ್ ವರ್ಗೀಕಾರಕವನ್ನು ಅನ್ವಯಿಸಲು, ನಾವು `TimeDistributed` ರಚನೆಯನ್ನು ಬಳಸುತ್ತೇವೆ, ಇದು ಪ್ರತಿಯೊಂದು ಹಂತದಲ್ಲಿಯೂ LSTM ಔಟ್‌ಪುಟ್‌ಗಳಿಗೆ ಅದೇ ಡೆನ್ಸ್ ಲೇಯರ್ ಅನ್ನು ನಕಲಿಸುತ್ತದೆ:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

ಇಲ್ಲಿ ಗಮನಿಸಿ ನಾವು ನಮ್ಮ ಡೇಟಾಸೆಟ್‌ಗೆ ಸ್ಪಷ್ಟವಾಗಿ `maxlen` ಅನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸುತ್ತಿದ್ದೇವೆ - ನೆಟ್‌ವರ್ಕ್ ಬದಲಾಗುವ ಉದ್ದದ ಕ್ರಮಗಳನ್ನು ನಿರ್ವಹಿಸಲು ಸಾಧ್ಯವಾಗಬೇಕಾದರೆ, ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವಾಗ ಸ್ವಲ್ಪ ಹೆಚ್ಚು ಚತುರವಾಗಿರಬೇಕಾಗುತ್ತದೆ.

ಈಗ ನಾವು ಮಾದರಿಯನ್ನು ತರಬೇತಿಗೆ ಒಳಪಡಿಸೋಣ. ವೇಗಕ್ಕಾಗಿ, ನಾವು ಕೇವಲ ಒಂದು ಎಪೋಕ್‌ಗೆ ತರಬೇತಿ ನೀಡುತ್ತೇವೆ, ಆದರೆ ನೀವು ಹೆಚ್ಚು ಸಮಯ ತರಬೇತಿ ನೀಡಲು ಪ್ರಯತ್ನಿಸಬಹುದು. ಜೊತೆಗೆ, ನೀವು ಡೇಟಾಸೆಟ್‌ನ ಕೆಲವು ಭಾಗವನ್ನು ತರಬೇತಿ ಡೇಟಾಸೆಟ್ ಆಗಿ ವಿಭಜಿಸಿ, ಮಾನ್ಯತೆ ನಿಖರತೆಯನ್ನು ಗಮನಿಸಲು ಬಯಸಬಹುದು.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## ಫಲಿತಾಂಶವನ್ನು ಪರೀಕ್ಷಿಸುವುದು

ನಾವು ಈಗ ನಮ್ಮ ಘಟಕ ಗುರುತಿಸುವ ಮಾದರಿ ಒಂದು ಉದಾಹರಣಾ ವಾಕ್ಯದಲ್ಲಿ ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ ಎಂದು ನೋಡೋಣ:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## ಸಾರಾಂಶ

ಸರಳ LSTM ಮಾದರಿಯೂ NER ನಲ್ಲಿ ಸಮರ್ಪಕ ಫಲಿತಾಂಶಗಳನ್ನು ತೋರಿಸುತ್ತದೆ. ಆದರೆ, ಉತ್ತಮ ಫಲಿತಾಂಶಗಳನ್ನು ಪಡೆಯಲು, BERT ಮುಂತಾದ ದೊಡ್ಡ ಪೂರ್ವ-ಪ್ರಶಿಕ್ಷಿತ ಭಾಷಾ ಮಾದರಿಗಳನ್ನು ಬಳಸಬಹುದು. Huggingface Transformers ಗ್ರಂಥಾಲಯವನ್ನು ಬಳಸಿ BERT ಅನ್ನು NER ಗೆ ತರಬೇತುಗೊಳಿಸುವುದು [ಇಲ್ಲಿ](https://huggingface.co/course/chapter7/2?fw=pt) ವಿವರಿಸಲಾಗಿದೆ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜು [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ಧತೆಯತ್ತ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
